In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: exogeneidad 

# Importando librerias:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

In [ ]:
Xeduc = datos['educacion']
Xposg = datos['edad']
Xante = datos['antiguedad_ejecutivo']
ylogs = np.log(datos['salario'])

X = np.column_stack((Xeduc, Xposg, Xante))
X = pd.DataFrame(X, columns=['educacion', 'edad', 'antiguedad'])
X = sm.add_constant(X)
X.columns = ['constante', 'educacion', 'edad', 'antiguedad'] 
modelo_mco = sm.OLS(ylogs,X).fit()
print(modelo_mco.summary())

In [ ]:
# Evaluación del instrumento
Zedum = datos['educacionmadre'] 
Z = sm.add_constant(Zedum)
modelo_mco_paso1 = sm.OLS(Xeduc, Z).fit()
print(modelo_mco_paso1.summary())
educacion_ajustada = modelo_mco_paso1.fittedvalues
residuos_mco_paso1 = modelo_mco_paso1.fittedvalues

In [ ]:
# Modelo VI (segundo paso de MCO 2 pasos)
Z = educacion_ajustada
ZX = np.column_stack((Z, Xposg, Xante))
ZX = pd.DataFrame(ZX, columns=['educacion', 'edad', 'antiguedad'])
ZX = sm.add_constant(ZX)
ZX.columns = ['constante', 'educacion', 'edad', 'antiguedad'] 
modelo_VI = sm.OLS(ylogs,ZX).fit()
print(modelo_VI.summary())

In [ ]:
import numpy as np
from scipy import stats
# Funcion para calcular el test de Hausman
def test_hausman_test(mco, vi):
    b = mco.params
    B = vi.params
    v_b = mco.cov_params()
    v_B = vi.cov_params()
    df = len(b)
    diff = b - B
    stat = np.dot(np.dot(diff.T, np.linalg.inv(v_b - v_B)), diff)
    pval = stats.chi2.sf(stat, df)
    return stat, pval

# Testeando exogeneidad:
hausman_stat, hausman_pval = test_hausman_test(modelo_mco, modelo_VI)
print(f'Estadígrafo de Hausman: {hausman_stat}')
print(f'Valor p de Hausman: {hausman_pval}')

if hausman_pval < 0.05:
    print("Rechazamos la hipótesis nula de exogeneidad. Existe evidencia de endogeneidad.")
else:
    print("No rechazamos la hipótesis nula de exogeneidad. No hay evidencia de endogeneidad.")
